# L4 – Word embedding

### Resources
1. Possible [truncated svd](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) implementation of sklearn.
2. Tensorflow [tutorial](https://www.tensorflow.org/tutorials/word2vec) for word2vec and [implementation](https://github.com/tensorflow/models/tree/master/tutorials/embedding).
3. GloVe [site](https://nlp.stanford.edu/projects/glove/).
4. FastText [site](https://fasttext.cc).
5. [Gensim](https://radimrehurek.com/gensim/) library.

**Word embedding** – the collective name for a set of language modeling and feature learning techniques in natural language processing where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with much higher dimensionality. Word and phrase embeddings, when used as the underlying input representation, have been shown to boost the performance in NLP tasks such as syntactic parsing, sentiment analysis and translation. Usually, problem reduces to unsupervised learning on a large corpus of text. So, learning process is based on the idea that context of word closely associated with it. For example, the context can be a document in which the word is located or adjacent words.

In [1]:
import gensim as gs
from gensim.corpora import WikiCorpus, MmCorpus, Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec, FastText

import numpy as np

import logging

import multiprocessing

from collections import defaultdict

import scipy as sp
from scipy.spatial.distance import cdist

from sklearn.decomposition import TruncatedSVD

import tensorflow as tf

from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
logger = logging.getLogger()

## 1. Data

As text corpus you may use simple english [wikipedia](https://simple.wikipedia.org/wiki/Simple_English_Wikipedia).
1. 131K articles
2. 2K common English words.

Also you can use just english [wikipedia](https://en.wikipedia.org/wiki/Wikipedia):
1. Includes 5М+ articles
2. 2M+ different tokens

#### Exercises
1. Download all wikipedia articles.
2. Make all preprocessing work that you need (e.g. remove markup).

In [ ]:
wiki = WikiCorpus("../lab_4/simplewiki-20170820-pages-meta-current.xml.bz2")
wiki.dictionary.filter_extremes(no_below=5, no_above=0.3)

In [ ]:
wiki.save('wiki')

In [ ]:
wiki = WikiCorpus.load('wiki')

### 2. LSA (Latent semantic analysis)
This solution uses full document as a context of word. So, we have some vocabulary $W$ and a set of documents $D$. Matrix $X$ with shape $|W| \times |D|$ at position $w, d$ stores importance of word $w$ for document $d$. If word $w$ is not found in the document $d$ than at appropriate position $X$ has 0 (obviously, matrix is sparse).

For each matrix you can find [SVD decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
$$X = U \Sigma V^{T} \text{, где }$$
* $U$ – orthogonal matrix $|W| \times |W|$ of left singular vectors
* $\Sigma$ – diagonal matrix $|W| \times |D|$ of singular values
* $V$ – orthogonal matrix $|D| \times |D|$  of right singular vectors

Let's suppouse that row $w$ in matrix $U\Sigma$ is a vector that represents word $w$, and row $d$ of $V$ coresponds to document $d$. In some sense we already found the embeddings of words and documents at the same time. But size of vectors are determined by documents number $|D|$.

Nevertheless you can use truncated SVD instead
$$ X \approx X_k = U_k \Sigma_k V^{T}_k \text{, where }$$
* $U_k$ – $k$ left singular vectors
* $\Sigma_k$ – diagonal matrix $|k| \times |k|$ of largest singular values
* $V_k$ – $k$ right singular vectors

Also it known that $X_k$ is the best approximation of $X$ in the term of [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm) for all matrices of rank $k$.

So, it is possible to obtain a compressed words' representations of size $k \ll |W|$ as rows of matrix $U_k \Sigma_k $. It just remains to understand how to calculate the original values of the matrix $X$. There is a set of approaches, here are some of them
1. Binary flag that document contains the word.
2. The number occurrences of word in document.
3. Word frequency that is better known as $tf$ (it's possible also use $\ln(1 + tf)$).
4. [$tf \cdot idf$](https://ru.wikipedia.org/wiki/TF-IDF).

#### Further readning
1. You also can read this [aticle](https://en.wikipedia.org/wiki/Latent_semantic_analysis).
2. By the way, some [modification](http://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) of SVD decompostion won at [Netflix prize](https://en.wikipedia.org/wiki/Netflix_Prize).
3. It is easy to see that the problem of SVD decomposition is reduced to finding eigenvectors of matrix $X \cdot X^T$.
4. You can find [here](https://arxiv.org/abs/0909.4061) probabilistic algorithms of matrix decomposition.

#### Exercises
1. Find matrix $X$, using you favourite approach (maybe you need the sparse matrix class of scipy library).
2. Find word embeddings of size $k = 128$.
3. Implement k-nearest neighbor search for Euclidean norm.
4. Find 10 nearest words for the set of words: 'cat', 'loves', 'clever' and 'mandarin'.
5. Repeat experiment for cosinus as metric. What are the results?

In [3]:
tfidf = TfidfModel(corpus=wiki, id2word=wiki.dictionary, normalize=True)

In [10]:
mod_svd = TruncatedSVD(n_components=128)
svd_emb = mod_svd.fit_transform(gs.matutils.corpus2csc(tfidf[wiki], 
                                                       num_terms=len(wiki.dictionary),
                                                       num_docs=wiki.length))

In [ ]:
%reset_selective tfidf

In [8]:
def find_neighbs(word, data, word_to_vec, ind_word, metric="Euclidean", k=10):
    vec = word_to_vec(word).reshape(1, data.shape[1])
    distances = cdist(data, vec, metric=metric).flatten()
    return [ind_word(i) for i in np.array(distances).argsort()[:k + 1] if ind_word(i) != word][:k]

def KNN(word_list, data, word_to_vec, ind_word, metrics=['Euclidean'], k=10):    
    for i in range(len(word_list)):
        print("Top %d nearest words to %s:"%(k, word_list[i]))
        for j in range(len(metrics)):
            nbs = find_neighbs(word_list[i], data, word_to_vec, ind_word, metrics[j], k=k)
            print(metrics[j], ": ", ", ".join(nbs), sep="")
        print()

In [9]:
word_list = ['cat', 'loves', 'clever', 'mandarin', 'chechnya']

In [55]:
KNN(word_list, 
    svd_emb,
    lambda word: svd_emb[wiki.dictionary.doc2bow([word])[0][0]],
    lambda ind: wiki.dictionary[ind],
    ["Euclidean", "Cosine"],
    k=10)

Top 10 nearest words to cat:
Euclidean: cats, breed, dogs, fur, bear, spider, dog, mouse, pet, nest
Cosine: cats, kittens, pet, dog, feline, tabby, furry, ears, paws, pets

Top 10 nearest words to loves:
Euclidean: likes, funny, mom, ugly, wonderful, curse, witch, dreams, wish, magical
Cosine: happy, friends, ugly, friend, mom, witch, wonderful, girl, happily, likes

Top 10 nearest words to clever:
Euclidean: wakes, realize, frightened, bored, shocked, disguise, honest, kindness, disguised, persuade
Cosine: really, telling, thinks, angry, knows, sees, sad, tell, frightened, bored

Top 10 nearest words to mandarin:
Euclidean: simplified, cantonese, taiwanese, guangdong, macau, manchu, chen, guangzhou, chiang, mongolian
Cosine: cantonese, simplified, hakka, chinese, pinyin, hanyu, guangdong, bai, taiwanese, uyghur

Top 10 nearest words to chechnya:
Euclidean: vasili, lyudmila, voronezh, nikolaevich, petrovich, aleksei, chechen, andropov, yury, nizhny
Cosine: russian, chechen, russia, yel

#### 3. word2vec
Let's consider perhaps the most popular model of word embeddings. This is mainly due to the fact that the received vectors have interesting properties. In particular, semantically similar words have close vectors, moreover linear operation of subtraction has meaning in the resulting vector space! The context of word at this time is a window of size $2c+1$, where interested word is located in the middle.

#### Continuous bag-of-words
The core idea is very simple. There is some very long text
$$w_1, w_2, \dots, w_T.$$
Consider some word $w_t$ and its context in radius $c$, namely words
$$w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}.$$
Intuition tells us that the context often really well defines the word in the middle. Then let our model restore the central word if its context is known
$$\mathcal{L} = - \frac{1}{T}\sum_{t} \log \mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}].$$

For each word $w_t$ there is some vector $v_t$, but if word $w_{t+i}$ is some context than we use vector $v'_{t+i}$. In this case we can represent context of word as the following sum
$$s'_t = \sum_{c \leq i \leq c, i \neq 0} v'_{t+i}$$

And define probabilty with softmax
$$\mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t-1}, \dots w_{t+c}] = \frac{ \exp {s'}_t^T \cdot v_t}{\sum_j \exp {s'}_t^T \cdot v_j}.$$

Note that this model is easy to present as 2-layer neural network:
* The input is a sparse vector of dimension $|W|$ which at position $w'$ has one if word $w'$ is in context.
* Further, we have matrix $V'$ of vectors $v'$.
* Further, without any non-linearity matrix $V$ of vector $v$.
* And then the softmax layer.

#### Skip-gram
Another model predicts context for some word. The design is approximately the same, we need to optimize
$$\mathcal{L} = -\frac{1}{T} \sum_t \sum_{-c \leq i \leq c, i \neq 0} \mathbb{P}[w_{t+i}|w_t].$$

The probability is approximated by the following model
$$\mathbb{P}[w_{t+i}|w_t] = \frac{ \exp {v'}_{t+i}^T \cdot v_t}{\sum_j \exp {v'}_{j}^T \cdot v_t}.$$


#### Hierarchical softmax
Creation of this mode was guided to reduce the complexity of learning. However, computing of sofmax's denominator is really expensive operation. Therefore, in practice, a few other models are used.

This method was viewed in the seminar. Detailed information can be found [here](http://www.iro.umontreal.ca/~lisa/pointeurs/hierarchical-nnlm-aistats05.pdf) and practical using [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).    Briefly recall, for each word from the dictionary we define [Huffman code](https://en.wikipedia.org/wiki/Huffman_coding). More frequency words have shorter code. Then we construct the Huffman tree. In accordance with its code words are stored in leaves. In the non-leaf vertices are stored special hidden representations $h'$. 

$$\mathbb{P}[w_{t+i}|w_t] = \prod_{{h'}_{t+i}, r} \sigma\Big( \mathcal{r} \cdot {h'}^T_{t+i} v_t \Big) \text{, where}$$
* ${h'}_{t+i}$ - hidden vector of word's path $w_{t+i}$
* $\mathcal{r}$ - equals $-1$, if we turn left and $+1$ in another case
* $\sigma$ - sigmoid function

Using hierarchical softmax as approximation of softmax allows us to significantly reduce complexity of model training. 

#### Exercise
1. How can you implement skip-gram model as neural network.
2. Estimate complexity of one iteration of training the skip-gram model
  * $T$ - text size
  * $W$ - vocabulary size
  * $c$ - context radius
  * $d$ - embedding size
3. Estimate complexity of using hierarchical softmax in the skip-gram model.

#### Why sampling?
Let see at softmax funciton
$$\mathbb{P}[x_i] = \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)}$$
Optimizing this function is equivalent to optimizing its logarithm
$$\log \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)} = f(x_i|\theta) - log \sum_j \exp f(x_j|\theta)$$
Than gradient $\triangledown_{\theta} \mathbb{P}[x_i]$ is equal to
$$\triangledown_{\theta} f(x_i|\theta) - \sum_k \frac{\exp f(x_k|\theta)}{\sum_j \exp f(w_j|\theta)} \triangledown_{\theta} f(x_k|\theta)$$
Inside the sum we can find $\mathbb{P}[x_k]$ again. So, the previous expression can be written as
$$
\triangledown_{\theta} f(x_i|\theta) - \sum_k \mathbb{P}[x_k] \cdot \triangledown_{\theta} f(x_k|\theta) = 
\triangledown_{\theta} f(x_i|\theta) - \mathbb{E}_{x \sim \mathbb{P}[x]} \triangledown_{\theta} f(x|\theta)
$$
If we can sample $x \sim \mathbb{P}[x]$, then we has no sense to calculate the softmax itself. We just can train model, computing only gradient. It is possible due to the fact that we do not need the probability itself, but want to find representation of words only.

#### Negative Sampling
As the result, in practice, softmax loss functions is approximated with negative sampling, which looks like a series of logistic regressions
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t}
    \sum_{-c \leq i \leq c, i \neq 0} \Big(
        \mathbb{P}[y = 1 | w_{t+i}, w_t] -
        k \cdot \mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]} \mathbb{P}[y = 0 | \tilde{w}_{tj}, w_t]
    \Big)
$$
where $y=1$, if  word is a context of word and $y=0$ in another case. А $\tilde{w}_{tj}$ – sampling of words that don't belong to the context. Then main trick is how we will sample words $\mathbb{P}[y | w_{t+i}, w_t]$. In paper [noise contrastive estimation](https://www.cs.helsinki.fi/u/ahyvarin/papers/Gutmann10AISTATS.pdf) some interesting approaches is suggested. But we are going to use more simple variation
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t} \Big(
    \sum_{-c \leq i \leq c, i \neq 0} \big(
        \sigma({v'}_{t+i}^T v_t) +
        k \cdot \mathbb{E}_{w_j \sim \mathbb{Q}(w_j)} \sigma (-{v'}_{j}^T v_t)
    \big)
\Big),
$$
to find  $\mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]}$ we simply sample words $k$ times from distribution $\mathbb{Q}$, $\mathbb{P}(w_j|w_t) = \mathbb{Q}(w_j)$.

#### Additional word2vec tricks
For the best quality at the moment of learning it is proposed to use variety of tricks:
1. How to choose $\mathbb{Q}$.
2. Sampling of frequent and rare words.
3. The sampling of window size.
You are welcome to read about this in original papers.

#### Further reading
1. Briefly about both models [here](https://arxiv.org/pdf/1301.3781.pdf).
2. Hierarchical softmax and negative sampling [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

#### Exercises
1. Train skip-gram model, using negative sampling and embedding size $d$ = 256. You are free to choose size of window and number of epochs.
2. Find top-10 nearest words again. Compare Euclidean and cosinus metric. Which is better? 
3. Find 10 nearest vector for expression v(king) - v(man) + v(women). If your model is tuned well, you find representation of word queen. 
4. How could you solve the following problems?
  * Find for country its capital
  * For a word find its plural form
  * For word define antonyms

In [17]:
class MySentences(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for text in self.corpus.get_texts():
            yield text

In [18]:
sentences = MySentences(wiki)
params_w2v = {'size': 300,
              'window': 10,
              'min_count': 0,
              'workers': min(1, multiprocessing.cpu_count() - 1),
              'sample': 1e-3,
              'sg': 1}

In [ ]:
skip_gr = Word2Vec(sentences, **params_w2v)

In [ ]:
skip_gr.save('skip_gr')

In [ ]:
model_name = 'skip_gr'
skip_gr = gs.models.Word2Vec.load(model_name)

In [10]:
sk_gr_emb = skip_gr[skip_gr.wv.index2word]
KNN(word_list,
    sk_gr_emb,
    lambda word: skip_gr[word],
    lambda ind: skip_gr.wv.index2word[ind],
    ["Euclidean", "Cosine"],
    k=10)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


Top 10 nearest words to cat:


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


Euclidean: cats, nyan, ocicat, darn, shorthair, fanciers, korat, fraidy, felis, poodle
Cosine: cats, felis, nyan, tabby, chaus, ocicat, darn, shorthair, fanciers, temminckii

Top 10 nearest words to loves:
Euclidean: despises, nannetta, dismay, imagines, annoys, chulsu, crowfeather, motherly, snobbish, tomboyish
Cosine: crowfeather, despises, flirts, nannetta, chulsu, sillier, aburatsubo, hates, khokhlakov, snobbish

Top 10 nearest words to clever:
Euclidean: unkind, thoughtful, likeable, respectable, obedient, snobbish, flirtatious, unselfish, funnier, kolya
Cosine: thoughtful, unkind, snobbish, likeable, pompous, respectable, obedient, rakitin, unselfish, funnier

Top 10 nearest words to mandarin:
Euclidean: cantonese, hokkien, jyutping, shanghainese, hlai, teochew, singlish, xing, hakka, jawi
Cosine: cantonese, shanghainese, teochew, hokkien, jyutping, hlai, hakka, singlish, hànzì, 漢字

Top 10 nearest words to chechnya:
Euclidean: dagestan, ingushetia, nikolayevsk, akhmad, umarov, vi

In [11]:
skip_gr.wv.most_similar(positive=['woman', 'king'], negative=['man'])

2018-02-10 12:18:18,821 : INFO : precomputing L2-norms of word weight vectors


[('queen', 0.5624725818634033),
 ('regnant', 0.5296790599822998),
 ('æthelred', 0.5111878514289856),
 ('unready', 0.4998839795589447),
 ('harthacnut', 0.4959748089313507),
 ('ancestress', 0.49359387159347534),
 ('yejong', 0.49324536323547363),
 ('athénaïs', 0.4912017583847046),
 ('betrothed', 0.48996302485466003),
 ('aelfwynn', 0.4896272122859955)]

In [12]:
skip_gr.wv.most_similar(positive=['russia', 'berlin'], negative=['germany'])

[('moscow', 0.6750845909118652),
 ('russian', 0.5874471664428711),
 ('leningrad', 0.569296658039093),
 ('kirov', 0.544143795967102),
 ('novosibirsk', 0.5334804058074951),
 ('bolshoi', 0.52711021900177),
 ('izhevsk', 0.5215718746185303),
 ('pushkin', 0.52144855260849),
 ('gusev', 0.520604133605957),
 ('soviet', 0.5204828977584839)]

In [13]:
skip_gr.wv.most_similar(positive=['book', 'trees'], negative=['tree'])

[('books', 0.6372838020324707),
 ('published', 0.5794482827186584),
 ('winesburg', 0.5573920011520386),
 ('autobiographies', 0.5557173490524292),
 ('bestsellers', 0.554986298084259),
 ('serialization', 0.5518832802772522),
 ('muckrakers', 0.5479726195335388),
 ('newsstands', 0.5469224452972412),
 ('ludvigsen', 0.5446893572807312),
 ('aphorisms', 0.544371485710144)]

In [14]:
skip_gr.wv.most_similar(positive=['black', 'king'], negative=['queen'])

[('white', 0.5163386464118958),
 ('canadieans', 0.40941116213798523),
 ('eyed', 0.40701964497566223),
 ('dark', 0.4062197208404541),
 ('brown', 0.4031407833099365),
 ('supremacists', 0.40268000960350037),
 ('submarginal', 0.3947872519493103),
 ('kimba', 0.3941536545753479),
 ('hawks', 0.39012351632118225),
 ('leswicks', 0.3865800201892853)]

### 4. GloVe
This model combines two ideas:
1. Factorization of the matrix, as in LSA
2. Context of word is adjacent words in the corpus, just in word2vec.

Let matrix $X$ at position $i,j$ stores how many times the word $j$ was found in the context of word $i$. This matrix is filled with our corpus of text. Then the probability that the word $j$ appears in the context of word $i$ is equal to
$$P_{ij} = \frac{X_{ij}}{\sum_k X_{ik}}$$

At the same time we want to find some function $F$ and word embeddings that 
$$F((w_i - w_j)^T w_k) = \frac{F(w_i^T w_k)}{F(w_j^T w_k)} = \frac{P_{ik}}{P_{jk}}.$$
About motivation it's better to read in the original [paper](http://nlp.stanford.edu/pubs/glove.pdf).
Possible solution may be following
$$\exp(w_i^T w_j) = P_{ij}.$$

After a series of transformations you can receive that
$$w_i^T w_j = \log X_{ij} - \log\big(\sum_k X_{ik}\big)$$

Obviously, right part doesn't depend on $j$, but the resulting model is offered to be written symmetry as
$$w_i^T w_j + b_i + b_j = \log X_{ij}$$

As loss function authors suggest to use weighted MSE
$$\mathcal{L} = \sum_{i,j = 1}^{|W|} f(X_{ij}) \Big(w_i^T w_j + b_i + b_j - \log(X_{ij}) \Big)^2,$$
where $f$ is some weight function, defined for each pair of words. It is reasonable to store $X$ as sparse matrix. It is also noteworthy that during training only **not null** values of $X$ used. Model is trained with stochastic gradient descent. As some way of regularization the authors suggest to use two different matrices $W$ и $W'$ for word and context, so model can be rewritten as
$$w_i^T {w'}_j + b_i + b_j = \log X_{ij}$$
The resulted embedding of a word is the sum of two trained views $w + w'$.

#### Some additional GloVe tricks
1. In order to understand how you can choose $f$, please, read the original paper.
2. Also you are welcome to see, how $X$ is formed in the original article.

#### Further reading
1. GloVe [site](http://nlp.stanford.edu/projects/glove/).
2. Some words about [t-SNE](http://lvdmaaten.github.io/tsne/).

#### Exercises
1. Estimate complexity of model for one iteration. Choose appropriate $c$ for you.
2. Train GloVe word embeddings ($d$=256).
3. Check that v(king) - v(man) + v(women) is approximately equal v(queen).
4. Read about [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).
5. Use t-SNE to reduce the size of embeddings to 3. Make sure that the following groups of vectors are collinear (use visualization)
  * [man, woman], [Mr., Ms], [king, queen], etc
  * [CEO, company]
  * [adjective, its comparative form]

In [5]:
class GloVeModel():
    def __init__(self, embedding_size, context_size, max_vocab_size=100000,
                 scaling_factor=3/4, cooccurrence_cap=150, batch_size=256, learning_rate=0.05):
        self.embedding_size = embedding_size
        self.left_context = self.right_context = context_size
        
        self.max_vocab_size = max_vocab_size
        self.scaling_factor = scaling_factor
        self.cooccurrence_cap = cooccurrence_cap
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.__words = None
        self.__word_to_id = None
        self.__id_to_word = None
        self.__cooccurrence_matrix = None
        self.__embeddings = None

    def fit_to_corpus(self, corpus, vocabulary, dictionary=[], reversed_dictionary=[]):
        self.vocab_size = len(dictionary)
        self.__fit_to_corpus(corpus, vocabulary, dictionary, self.max_vocab_size,
                             self.left_context, self.right_context)
        self.__id_to_word = reversed_dictionary
        logger.info('Building tensorflow graph')
        self.__build_graph()

    def __fit_to_corpus(self, corpus, vocabulary, dictionary, vocab_size, left_size, right_size):
        cooccurrence_counts = defaultdict(float)
        logger.info('Processing corpus to find cooccurence values of words')
        for region in tqdm(corpus.get_texts(), total=corpus.length):
            filt_region = [word for word in region if word in vocabulary]
            for l_context, word, r_context in _context_windows(filt_region, left_size, right_size):
                for i, context_word in enumerate(l_context[::-1]):
                    cooccurrence_counts[(word, context_word)] += 1 / (i + 1)
                for i, context_word in enumerate(r_context):
                    cooccurrence_counts[(word, context_word)] += 1 / (i + 1)
            
        self.__words = vocabulary
        self.__word_to_id = dictionary
        logger.info('Constructing cooccurence matrix')
        self.__cooccurrence_matrix = {
            (dictionary[words[0]], dictionary[words[1]]): count
            for words, count in tqdm(cooccurrence_counts.items())}

    def __build_graph(self):
        self.__graph = tf.Graph()
        with self.__graph.as_default():
            count_max = tf.constant([self.cooccurrence_cap], dtype=tf.float32,
                                    name='max_cooccurrence_count')
            scaling_factor = tf.constant([self.scaling_factor], dtype=tf.float32,
                                         name="scaling_factor")

            self.__focal_input = tf.placeholder(tf.int32, shape=[self.batch_size],
                                                name="focal_words")
            self.__context_input = tf.placeholder(tf.int32, shape=[self.batch_size],
                                                  name="context_words")
            self.__cooccurrence_count = tf.placeholder(tf.float32, shape=[self.batch_size],
                                                       name="cooccurrence_count")

            focal_embeddings = tf.Variable(
                tf.random_uniform([self.vocab_size, self.embedding_size], 1.0, -1.0),
                name="focal_embeddings")
            context_embeddings = tf.Variable(
                tf.random_uniform([self.vocab_size, self.embedding_size], 1.0, -1.0),
                name="context_embeddings")

            focal_biases = tf.Variable(tf.random_uniform([self.vocab_size], 1.0, -1.0),
                                       name='focal_biases')
            context_biases = tf.Variable(tf.random_uniform([self.vocab_size], 1.0, -1.0),
                                         name="context_biases")

            focal_embedding = tf.nn.embedding_lookup([focal_embeddings], self.__focal_input)
            context_embedding = tf.nn.embedding_lookup([context_embeddings], self.__context_input)
            focal_bias = tf.nn.embedding_lookup([focal_biases], self.__focal_input)
            context_bias = tf.nn.embedding_lookup([context_biases], self.__context_input)

            weighting_factor = tf.minimum(
                1.0,
                tf.pow(
                    tf.div(self.__cooccurrence_count, count_max),
                    scaling_factor))

            embedding_product = tf.reduce_sum(tf.multiply(focal_embedding, context_embedding), 1)

            log_cooccurrences = tf.log(tf.to_float(self.__cooccurrence_count))

            distance_expr = tf.square(tf.add_n([
                embedding_product,
                focal_bias,
                context_bias,
                tf.negative(log_cooccurrences)]))

            single_losses = tf.multiply(weighting_factor, distance_expr)
            self.__total_loss = tf.reduce_sum(single_losses)
            self.__optimizer = tf.train.AdagradOptimizer(self.learning_rate).minimize(
                self.__total_loss)

            self.__combined_embeddings = tf.add(focal_embeddings, context_embeddings,
                                                name="combined_embeddings")

    def train(self, num_epochs, loss_interval=50000):
        logger.info('Preparing batches')
        batches = self.__prepare_batches()
        total_steps = 0
        avg_loss = 0
        with tf.Session(graph=self.__graph) as session:
            logger.info('Initializing variables')
            tf.global_variables_initializer().run()
            for epoch in range(num_epochs):
                logger.info('Epoch #%d' % (epoch + 1))
                shuffle(batches)
                for batch_index, batch in enumerate(batches):
                    i_s, j_s, counts = batch
                    if len(counts) != self.batch_size:
                        continue
                    feed_dict = {
                        self.__focal_input: i_s,
                        self.__context_input: j_s,
                        self.__cooccurrence_count: counts}
                    loss, _ = session.run([self.__total_loss, self.__optimizer], feed_dict=feed_dict)
                    avg_loss += loss
                    if total_steps % loss_interval == 0:
                        if total_steps > 0:
                            avg_loss /= loss_interval
                            print('Average loss at step ', total_steps, ': ', avg_loss)
                        avg_loss = 0
                    total_steps += 1
            logger.info('Finished training')
            self.__embeddings = self.__combined_embeddings.eval()


    def __prepare_batches(self):
        if self.__cooccurrence_matrix is None:
            raise NotFitToCorpusError(
                "Need to fit model to corpus before preparing training batches.")
        cooccurrences = [(word_ids[0], word_ids[1], count)
                         for word_ids, count in tqdm(self.__cooccurrence_matrix.items())]
        i_indices, j_indices, counts = zip(*cooccurrences)
        return list(_batchify(self.batch_size, i_indices, j_indices, counts))

    @property
    def embeddings(self):
        if self.__embeddings is None:
            raise NotTrainedError("Need to train model before accessing embeddings")
        return self.__embeddings
    
def _context_windows(region, left_size, right_size):
    for i, word in enumerate(region):
        start_index = i - left_size
        end_index = i + right_size
        left_context = _window(region, start_index, i - 1)
        right_context = _window(region, i + 1, end_index)
        yield (left_context, word, right_context)

def _window(region, start_index, end_index):
    last_index = len(region) + 1
    selected_tokens = region[max(start_index, 0):min(end_index, last_index) + 1]
    return selected_tokens

def _batchify(batch_size, *sequences):
    for i in range(0, len(sequences[0]), batch_size):
        yield tuple(sequence[i:i+batch_size] for sequence in sequences)

In [6]:
glove = GloVeModel(256, 5)

In [ ]:
glove.fit_to_corpus(wiki,
                    list(wiki.dictionary.itervalues()))

### 5. FastText
[FastText](https://arxiv.org/pdf/1607.04606.pdf) is a logical development of skip-gram model. The core feature is to present the word as the sum of its n-grams embeddings and its own embedding vector. For example, if we want to use only 3-grams, than word **fast** be reprented as n-grams **-fa**, **fas**, **ast**, **st-** and word **-fast-**. The context is encoeded in usual way, as a result similarity of word and context word is the following sum
$$s(w, c) = \sum_{n \in N_w} z^{T}_n v'_{c},$$
where $N_w$ is the set of all n-grams of word $w$ combined with word itself. The authors argue that the use of combination of 3-,4- and 5-grams helps to get better embeddings for rare and even unknown words (emdedding of word itself is null vector). For model training is proposed to use negative sampling.

You can find more information on [site](https://fasttext.cc) of FastText.

#### Exercises
1. Train FastText word embeddings ($d$=256).
2. Find some rare words in your corpus and find top-5 nearest words for them, using cosinus as metric.
3. Compare results with classic word2vec.
4. Invent some funny new word and found its top-5 nearest words again.
5. How could you compare all models. Suggest your metric (see papers for inspiration).

In [16]:
params_fasttext = {'size': 300,
                   'window': 10,
                   'min_count': 0,
                   'workers': min(1, multiprocessing.cpu_count() - 1),
                   'sample': 1e-3,
                   'sg': 1,
                   'word_ngrams': 1,
                   'min_n': 3}

In [ ]:
fast_text = FastText(sentences, **params_fasttext)

In [ ]:
fast_text.save('fast_text')

In [ ]:
model_name = 'fast_text'
fast_text = gs.models.FastText.load(model_name)

In [28]:
words = ['cat', 'loves', 'clever', 'mandarin']

In [32]:
fast_text.wv.similar_by_vector('cat')

[('cats', 0.7647467851638794),
 ('catgut', 0.7357016205787659),
 ('caty', 0.7125818133354187),
 ('catm', 0.6856584548950195),
 ('catv', 0.6698700189590454),
 ('ycat', 0.6695582270622253),
 ('catyo', 0.6676708459854126),
 ('catle', 0.6633636951446533),
 ('catmon', 0.6559504866600037),
 ('catdog', 0.6555511355400085)]

In [33]:
fast_text.wv.similar_by_vector('loves')

[('loved', 0.8175173997879028),
 ('likes', 0.7931973934173584),
 ('lovespring', 0.7864671945571899),
 ('lovers', 0.7844809293746948),
 ('lover', 0.7801826000213623),
 ('lovesong', 0.7780148386955261),
 ('lovestory', 0.7760759592056274),
 ('kloves', 0.7728105783462524),
 ('oogieloves', 0.7670139074325562),
 ('love涙色', 0.7669463753700256)]

In [34]:
fast_text.wv.similar_by_vector('clever')

[('cleverer', 0.8495073318481445),
 ('cleverly', 0.7978258728981018),
 ('cleverest', 0.7599146366119385),
 ('cleve', 0.7422327399253845),
 ('hasenclever', 0.7413553595542908),
 ('cleves', 0.735133945941925),
 ('cleverness', 0.7214470505714417),
 ('whomever', 0.7147210836410522),
 ('behaved', 0.7123929858207703),
 ('everyething', 0.7014926671981812)]

In [41]:
fast_text.wv.similar_by_word('rubachev')

[('gorbachev', 0.8419737815856934),
 ('bachev', 0.824567437171936),
 ('gorbacheva', 0.7680365443229675),
 ('gachev', 0.7550765872001648),
 ('khruschev', 0.7518523931503296),
 ('krushchev', 0.7457911372184753),
 ('grachev', 0.741490364074707),
 ('sergachev', 0.733389139175415),
 ('djachev', 0.7276131510734558),
 ('khrushchev', 0.7237513065338135)]